In [ ]:
import pandas as pd

In [ ]:
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.columns

Index(['Case Name', 'Input', 'Output', 'Label', 'Count', 'Decision_Count'], dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12178 entries, 0 to 12177
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Case Name       12178 non-null  object
 1   Input           12178 non-null  object
 2   Output          12178 non-null  object
 3   Label           12178 non-null  int64 
 4   Count           12178 non-null  int64 
 5   Decision_Count  12178 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 571.0+ KB


In [ ]:
df.head()

,Case Name,Input,Output,Label,Count,Decision_Count
0,"KAMLESH Vs. UNION OF INDIA THROUGH SECRETARY, ...","S. ABDUL NAZEER, J. 1. In this appeal, appella...",0[ds]7. It is clear from the order of appointm...,0,1253,564
1,KANWAR PAL SINGH Vs. THE STATE OF UTTAR PRADESH,"SANJIV KHANNA, J. 1. Leave granted. 2. Kanwar ...",1[ds]5. We find the submission of the appellan...,1,4065,2268
2,Manke Ram Vs. State of Haryana,"Santosh Hegde, J. 1. The incident which gives ...",1[ds]6. Having perused the material on record ...,1,1616,708
3,Kr. Jyoti Sarup and Another Vs. Board of Reven...,"S.K. Das, J. 1. These two appeals and the conn...","0[ds]7. We are in agreement with the view, exp...",0,2440,784
4,"Commissioner Of Income-Tax, Kerala Vs. Gemini ...","Shah, J. 1. Two persons-Walter and Ramasubramo...",1[ds]That case can have no application to the ...,1,3317,722


In [ ]:
def preprocess_case(text):
  max_tokens = 1000
  tokens = text.split(' ')
  num_tokens_to_extract = min(max_tokens, len(tokens))
  text1 = ' '.join(tokens[-num_tokens_to_extract:len(tokens)])
  return text1

In [ ]:
for i,row in df.iterrows():
  input = row['Input']
  input = preprocess_case(input)
  df.at[i,'Input'] = input

In [ ]:
#prediction and explanation
def format_instruction(sample, inst):
	return f"""### Instruction:
{inst}

### Input:
{sample['Input']}

### Response:
{sample['Output'].split('[ds]')[0]}

### Explanation:
{preprocess_case(sample['Output'].split('[ds]')[1])}
"""
#prediction

# def format_instruction(sample, inst):
# 	return f"""### Instruction:
# {inst}

# ### Input:
# {sample['Input']}

# ### Response:
# {sample['Output'].split('[ds]')[0]}
# """

In [ ]:
df_ins = pd.read_csv("instruction_decision.csv")

In [ ]:
import random
random.seed(15)
text = []
for i,row in tqdm(df.iterrows()):
  random_index = random.randint(0, len(df_ins) - 1)
  t = format_instruction(df.iloc[i],df_ins['Instructions_Exp'][random_index])
  #t = format_instruction(df.iloc[i],df_ins['Instructions'][random_index])
  text.append(t)

12178it [00:03, 3213.05it/s]


In [ ]:
df.loc[:, "text"] = text

In [ ]:
# Set a random state for reproducibility
random_state = 42

# Calculate the number of rows for validation set (10%)
val_size = int(0.1 * len(df))

df_val = df.sample(n=val_size, random_state=random_state)

df_train = df.drop(df_val.index)

df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [ ]:
df_train.to_csv("train_ft.csv",index=False)
df_val.to_csv("val_ft.csv",index=False)